In [1]:
from collections import Counter
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForMaskedLM

import torch

/home2/arihanth.srikar/miniconda3/envs/pytorch2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_60k", cache_dir='/scratch')
# model = AutoModelForMaskedLM.from_pretrained("seyonec/PubChem10M_SMILES_BPE_60k", cache_dir='/scratch')

tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MLM", cache_dir='/scratch')
model = AutoModelForMaskedLM.from_pretrained("DeepChem/ChemBERTa-77M-MLM", cache_dir='/scratch')

In [3]:
df = pd.read_pickle("data/uspto50/final_data.pickle")
df.head()

,reactants_mol,products_mol,reaction_type,set,num_reacts,num_prods,ratio,reactant_token,reactant_bfs_id,reactant_bfs_depth,reactant_dfs_id,reactant_dfs_depth,product_token,product_bfs_id,product_bfs_depth,product_dfs_id,product_dfs_depth
0,"[CS(=O)(=O)OC[C@H]1CCC(=O)O1, Fc1ccc(Nc2ncnc3c...",[O=C1CC[C@H](CN2CCN(CCOc3cc4ncnc(Nc5ccc(F)c(Cl...,<RX_1>,train,2,1,1.121951,"[[F, c, 1, c, c, c, (, N, c, 2, n, c, n, c, 3,...","[[0, 1, 1, 2, 32, 3, 3, 31, 33, 33, 4, 5, 6, 7...","[[0, 1, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 8, 8,...","[[0, 1, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 10, 1...","[[0, 1, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 8, 8,...","[[O, =, C, 1, C, C, [C@H], (, C, N, 2, C, C, N...","[[0, 0, 1, 1, 2, 40, 3, 3, 4, 5, 5, 6, 7, 39, ...","[[0, 0, 1, 1, 2, 3, 3, 3, 4, 5, 5, 6, 7, 8, 8,...","[[0, 0, 1, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 9,...","[[0, 0, 1, 1, 2, 3, 3, 3, 5, 6, 6, 7, 8, 9, 9,..."
1,[COC(=O)c1cc(CCCc2cc3c(=O)[nH]c(N)nc3[nH]2)cs1],[Nc1nc2[nH]c(CCCc3csc(C(=O)O)c3)cc2c(=O)[nH]1],<RX_6>,train,1,1,1.045455,"[[C, O, C, (, =, O, ), c, 1, c, c, (, C, C, C,...","[[0, 1, 2, 2, 2, 3, 3, 4, 4, 5, 22, 22, 6, 21,...","[[0, 1, 2, 2, 2, 3, 3, 3, 3, 4, 5, 5, 6, 7, 8,...","[[0, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 7, 8, 9,...","[[0, 1, 2, 2, 2, 3, 3, 3, 3, 4, 5, 5, 6, 7, 8,...","[[N, c, 1, n, c, 2, [nH], c, (, C, C, C, c, 3,...","[[0, 1, 1, 2, 21, 21, 3, 19, 19, 4, 18, 20, 5,...","[[0, 1, 1, 2, 3, 3, 4, 5, 5, 6, 7, 8, 9, 9, 10...","[[0, 1, 1, 2, 3, 3, 4, 5, 5, 6, 7, 8, 9, 9, 10...","[[0, 1, 1, 2, 3, 3, 4, 5, 5, 6, 7, 8, 9, 9, 10..."
2,"[CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C, FC(F)(F...",[CC1(C)OB(c2cccc(Nc3nccc(C(F)(F)F)n3)c2)OC1(C)C],<RX_9>,train,2,1,1.384615,"[[C, C, 1, (, C, ), O, B, (, B, 2, O, C, (, C,...","[[0, 1, 1, 1, 2, 2, 3, 15, 15, 4, 4, 14, 16, 1...","[[0, 1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 7,...","[[0, 1, 1, 1, 2, 2, 3, 4, 4, 5, 5, 6, 7, 7, 8,...","[[0, 1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 7,...","[[C, C, 1, (, C, ), O, B, (, c, 2, c, c, c, c,...","[[0, 1, 1, 1, 2, 2, 3, 23, 23, 4, 4, 22, 24, 2...","[[0, 1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 7, 6,...","[[0, 1, 1, 1, 2, 2, 3, 4, 4, 5, 5, 6, 7, 8, 9,...","[[0, 1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 7, 6,..."
3,[CC(C)(C)OC(=O)NCC(=O)CCC(=O)OCCCC(=O)OCc1ccccc1],[CC(C)(C)OC(=O)NCC(=O)CCC(=O)OCCCC(=O)O],<RX_6>,train,1,1,1.318182,"[[C, C, (, C, ), (, C, ), O, C, (, =, O, ), N,...","[[0, 1, 1, 2, 2, 2, 3, 3, 4, 5, 5, 5, 6, 6, 7,...","[[0, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4,...","[[0, 1, 1, 2, 2, 2, 3, 3, 4, 5, 5, 5, 6, 6, 7,...","[[0, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4,...","[[C, C, (, C, ), (, C, ), O, C, (, =, O, ), N,...","[[0, 1, 1, 2, 2, 2, 3, 3, 4, 5, 5, 5, 6, 6, 7,...","[[0, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4,...","[[0, 1, 1, 2, 2, 2, 3, 3, 4, 5, 5, 5, 6, 6, 7,...","[[0, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4,..."
4,"[Fc1cc2c(Cl)ncnc2cn1, NC1CCCCCC1]",[Fc1cc2c(NC3CCCCCC3)ncnc2cn1],<RX_1>,train,2,1,1.052632,"[[F, c, 1, c, c, 2, c, (, Cl, ), n, c, n, c, 2...","[[0, 1, 1, 2, 11, 11, 3, 3, 10, 10, 4, 9, 5, 6...","[[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 5, 4, 4,...","[[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 6, 7, 8, 9, 9,...","[[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 5, 4, 4,...","[[F, c, 1, c, c, 2, c, (, N, C, 3, C, C, C, C,...","[[0, 1, 1, 2, 18, 18, 3, 3, 17, 4, 4, 16, 5, 1...","[[0, 1, 1, 2, 3, 3, 4, 4, 5, 6, 6, 7, 8, 9, 9,...","[[0, 1, 1, 2, 3, 3, 4, 4, 5, 6, 6, 7, 8, 9, 10...","[[0, 1, 1, 2, 3, 3, 4, 4, 5, 6, 6, 7, 8, 9, 9,..."


In [4]:
df_train, df_val, df_test = df[df["set"] == "train"].copy(), df[df["set"] == "valid"].copy(), df[df["set"] == "test"].copy()

In [5]:
df_train['tokenized_reactants'] = df_train['reactants_mol'].apply(lambda x: [tokenizer(smi)['input_ids'] for smi in x])
df_train['tokenized_products'] = df_train['products_mol'].apply(lambda x: [tokenizer(smi)['input_ids'] for smi in x])

df_val['tokenized_reactants'] = df_val['reactants_mol'].apply(lambda x: [tokenizer(smi)['input_ids'] for smi in x])
df_val['tokenized_products'] = df_val['products_mol'].apply(lambda x: [tokenizer(smi)['input_ids'] for smi in x])

df_test['tokenized_reactants'] = df_test['reactants_mol'].apply(lambda x: [tokenizer(smi)['input_ids'] for smi in x])
df_test['tokenized_products'] = df_test['products_mol'].apply(lambda x: [tokenizer(smi)['input_ids'] for smi in x])

In [6]:
def convert_to_freq_map(data):
    frq_map = Counter()
    for row in tqdm(data):
        for product in row:
            temp_counter = Counter(product)
            frq_map.update(temp_counter)
    return frq_map

In [7]:
train_products_freq = convert_to_freq_map(df_train['tokenized_products'])
train_reactants_freq = convert_to_freq_map(df_train['tokenized_reactants'])

val_products_freq = convert_to_freq_map(df_val['tokenized_products'])
val_reactants_freq = convert_to_freq_map(df_val['tokenized_reactants'])

test_products_freq = convert_to_freq_map(df_test['tokenized_products'])
test_reactants_freq = convert_to_freq_map(df_test['tokenized_reactants'])

100%|██████████| 5004/5004 [00:00<00:00, 142548.68it/s]


In [8]:
train_reactants_set = set(train_reactants_freq.keys())
train_products_set = set(train_products_freq.keys())

val_reactants_set = set(val_reactants_freq.keys())
val_products_set = set(val_products_freq.keys())

test_reactants_set = set(test_reactants_freq.keys())
test_products_set = set(test_products_freq.keys())

In [15]:
print(len(val_reactants_set - train_reactants_set))
print(len(train_reactants_set - val_reactants_set), train_reactants_set - val_reactants_set)

0
1 {76}


In [16]:
tokenizer.vocab_size

591

In [ ]:
reactant_smi = df['reactants_mol'].iloc[109]
reactant_smi[0]

In [ ]:
gt_smi    = 'CC(C)(C)OC(=O)N1CCN(c2ccccc2OC2CCCNC2)CC1'
input_smi = 'CC(C)(C)<mask>(=O)N1CCN(c2ccccc2OC2CCCNC2)CC1'
# input_smi = 'CC(C)(C)[MASK]C(=O)N1CCN(c2ccccc2OC2CCCNC2)CC1'

In [ ]:
print(tokenizer(gt_smi)['input_ids'])
print(tokenizer(input_smi)['input_ids'])

In [ ]:
print([tokenizer.decode(t) for t in tokenizer(gt_smi)['input_ids']])
print([tokenizer.decode(t) for t in tokenizer(input_smi)['input_ids']])

In [ ]:
tokenizer.all_special_tokens, tokenizer.vocab_size

In [ ]:
print(model)

In [ ]:
model.config

In [ ]:
input_ids = tokenizer(input_smi, return_tensors="pt")
gt_ids    = tokenizer(gt_smi, return_tensors="pt")
# output = model(input_ids['input_ids'], labels=gt_ids['input_ids'], output_hidden_states=True)
output = model(input_ids['input_ids'], labels=torch.where(input_ids.input_ids == tokenizer.mask_token_id, gt_ids.input_ids, -100), output_hidden_states=True)

In [ ]:
model.roberta.encoder.layer[0].attention.self.query.weight

In [ ]:
torch.where(input_ids.input_ids == tokenizer.mask_token_id, gt_ids.input_ids, -100)

In [ ]:
output.keys()

In [ ]:
output['loss'].item(), output['logits'].shape

In [ ]:
output['hidden_states'] = torch.cat(output['hidden_states'], dim=0)

In [ ]:
output['hidden_states'].shape

In [ ]:
output['hidden_states'][0]

In [ ]:
input_ids['input_ids'].shape, output['logits'].shape

In [ ]:
special_tokens_dict = {'additional_special_tokens': ['[1]', '[2]', '[3]', '[4]']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
tok_id = tokenizer.convert_tokens_to_ids(['[1]', '[2]', '[3]', '[4]'])
tok_id